In [1]:
import os
import time
import math
import requests
import bs4 as bs
import numpy as np
import pandas as pd
import pickle as pkl
import datetime as dt
import tensorflow as tf
from sklearn import metrics
from matplotlib import style
import yahoo_finance as yahoo
from datetime import timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.finance import candlestick_ohlc
from IPython.display import display, Math, Latex
style.use('ggplot')

C:\Users\Khanh\Anaconda3\envs\py35\lib\site-packages\matplotlib\cbook.py:136: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [2]:
df = pd.read_csv('./ibm/full_ibm.csv')
df['Date.1'] = pd.to_datetime(df['Date.1'])
df.set_index('Date.1', inplace=True)
df.drop(['Date'], axis=1, inplace=True)
df.head()

,Open,High,Low,Close,Volume,Adj Close,NDX,DJIA,SP500
Date.1,,,,,,,,,
2000-01-03,112.4375,116.0000,111.875,116.0000,10347700,87.761136,3790.550049,11357.509766,1455.219971
2000-01-04,114.0000,114.5000,110.875,112.0625,8227800,84.782175,3546.199951,10997.929688,1399.420044
2000-01-05,112.9375,119.7500,112.125,116.0000,12733200,87.761136,3507.310059,11122.650391,1402.109985
2000-01-06,118.0000,118.9375,113.500,114.0000,7971900,86.248013,3340.810059,11253.259766,1403.449951
2000-01-07,117.2500,117.9375,110.625,113.5000,11856700,85.869732,3529.600098,11522.559570,1441.469971


In [3]:
def calculate_technical_indicator(data):
    data['ewma_5'] = data['Close'].ewm(span=5).mean()
    data['ewma_22'] = data['Close'].ewm(span=22).mean()
    data['ewma_200'] = data['Close'].ewm(span=200).mean()
    data['MACD'] = data['Close'].ewm(span=12).mean() - data['Close'].ewm(span=26).mean()
    data['Lower band'] = data['Close'].rolling(window=20).mean() - 2*data['Close'].rolling(window=20).std()
    data['Upper band'] = data['Close'].rolling(window=20).mean() + 2*data['Close'].rolling(window=20).std()
    data['Middle band'] = data['Close'].rolling(window=20).mean()
    data['AO'] = ((data['High'] + data['Low'])/2).rolling(window=5).mean() - ((data['High'] + data['Low'])/2).rolling(window=34).mean()
    data['AC'] =  data['AO'] - data['AO'].rolling(window=5).mean()
    data['%R'] = (data['High'].max() - data['Close'])*100/(data['High'].max()-data['Low'].min())
    
    return data.dropna(axis=0, how='any')
   
df = calculate_technical_indicator(df)
print (df.shape)
df.head()

(4240, 19)


,Open,High,Low,Close,Volume,Adj Close,NDX,DJIA,SP500,ewma_5,ewma_22,ewma_200,MACD,Lower band,Upper band,Middle band,AO,AC,%R
Date.1,,,,,,,,,,,,,,,,,,,
2000-02-25,109.8750,109.8750,104.9375,108.000,10312200,81.794653,4178.580078,9862.120117,1333.359985,110.379006,113.930717,115.468485,-1.540179,107.384880,120.746370,114.065625,-5.621691,-2.680221,66.650193
2000-02-28,104.6250,106.5000,103.9375,104.500,8479900,79.143900,4162.129883,10038.650391,1348.050049,108.419337,113.086350,115.130536,-2.041213,105.835213,121.589787,113.712500,-7.055147,-2.867868,68.812154
2000-02-29,105.5625,105.5625,100.9375,102.750,10484900,77.818524,4266.939941,10128.309570,1366.420044,106.529558,112.163277,114.756874,-2.543629,103.966489,122.508511,113.237500,-8.208456,-2.701728,69.893135
2000-03-01,102.0000,105.5000,100.0625,100.250,10807800,75.925129,4309.009766,10137.929688,1379.189941,104.436372,111.101869,114.327720,-3.099331,101.875492,123.624508,112.750000,-9.078860,-2.306103,71.437394
2000-03-02,100.5000,105.4375,99.5000,103.125,11192900,78.102533,4234.259766,10164.919922,1381.760010,103.999248,110.392690,114.002693,-3.277844,100.547640,123.914860,112.231250,-9.984559,-1.994816,69.661497


In [4]:
## params
# n_layers = 2 # Number of LSTM cells for multi LSTM cells
temporal_windows = 30
n_features = df.shape[1]
hidden_size = 15 # lstm weight size

In [5]:
def prepare_rnn_data(data, temporal_windows):
    x = np.zeros((data.shape[0]-temporal_windows, temporal_windows, data.shape[1]))
    data_array = data.as_matrix()
    for i in range(x.shape[0]):
        x[i,:,:] = data_array[i:temporal_windows+i,:]
    return x
df_rnn = prepare_rnn_data(df, temporal_windows)
df_rnn.shape

(4210, 30, 19)

In [6]:
def reshape_target_array(data, horizons):
    
    l = np.zeros((data.shape[0] + horizons - 1, 1))
    for i in range(data.shape[0]):
        l[i] = (np.mean([data[i-j, k] for j, k in zip(range(horizons), range(horizons)) if (i >= j)]))
    for i in range(horizons-1):
        l[data.shape[0] + i] = data[-1,i+1]
    return l

In [7]:
horizons = 5
def prepare_data_target(data, horizons):
    cols = data.columns
    l = []
    for h in range(horizons):
        shift = data.shift(periods=-h, axis=0)
        shift.columns = [(cols[i] + '_{}'.format(h+1)) for i in range(len(cols))]
        l.append(shift)
    output = pd.concat(l, axis=1)
    output.dropna(axis=0, how='any', inplace=True)
    return output
df_target = prepare_data_target(df[['Close']], horizons)
df_target = df_target.loc[df.index[temporal_windows:]]
df_target.dropna(axis=0, how='any', inplace=True)

In [8]:
df_target.tail()

,Close_1,Close_2,Close_3,Close_4,Close_5
Date.1,,,,,
2016-12-19,166.679993,167.600006,167.330002,167.059998,166.710007
2016-12-20,167.600006,167.330002,167.059998,166.710007,167.139999
2016-12-21,167.330002,167.059998,166.710007,167.139999,166.190002
2016-12-22,167.059998,166.710007,167.139999,166.190002,166.600006
2016-12-23,166.710007,167.139999,166.190002,166.600006,165.990005


In [9]:
df.tail(10)

,Open,High,Low,Close,Volume,Adj Close,NDX,DJIA,SP500,ewma_5,ewma_22,ewma_200,MACD,Lower band,Upper band,Middle band,AO,AC,%R
Date.1,,,,,,,,,,,,,,,,,,,
2016-12-16,168.970001,169.110001,166.059998,166.729996,7120600,165.422021,4914.859863,19843.410156,2258.070068,167.027881,163.427546,154.989750,2.586425,157.928373,169.568626,163.748500,7.344617,0.624629,30.372475
2016-12-19,166.830002,167.259995,166.000000,166.679993,2955900,165.372411,4934.850098,19883.060547,2262.530029,166.911918,163.710367,155.106071,2.498308,158.327848,169.798151,164.063000,7.099763,0.029005,30.403362
2016-12-20,167.490005,168.250000,166.449997,167.600006,2174600,166.285207,4953.799805,19974.619141,2270.760010,167.141281,164.048597,155.230389,2.474191,158.394491,170.214508,164.304500,6.579558,-0.521612,29.835067
2016-12-21,166.250000,167.940002,165.250000,167.330002,3575700,166.017321,4948.910156,19941.960938,2265.179932,167.204188,164.333936,155.350783,2.405561,158.532147,170.542852,164.537500,5.750294,-1.080446,30.001849
2016-12-22,167.360001,168.229996,166.580002,167.059998,2802600,165.749434,4934.390137,19918.880859,2260.959961,167.156125,164.570985,155.467293,2.302839,158.811937,170.771063,164.791500,5.037853,-1.324565,30.168631
2016-12-23,167.000000,167.490005,166.449997,166.710007,1701200,165.402189,4940.020020,19933.810547,2263.790039,167.007419,164.756987,155.579160,2.168195,158.984878,170.955123,164.970000,4.496765,-1.296082,30.384822
2016-12-27,166.979996,167.979996,166.850006,167.139999,1397500,165.828809,4965.810059,19945.039062,2268.879883,167.051612,164.964205,155.694194,2.072298,159.043100,171.158900,165.101000,4.287883,-0.942588,30.119214
2016-12-28,167.289993,167.740005,166.000000,166.190002,1757500,164.886264,4926.290039,19833.679688,2249.919922,166.764409,165.070796,155.798629,1.897766,159.204593,171.263408,165.234000,3.827913,-0.852229,30.706031
2016-12-29,166.020004,166.990005,166.000000,166.600006,1663500,165.293051,4918.279785,19819.779297,2249.260010,166.709608,165.203771,155.906106,1.772104,159.568089,171.337912,165.453001,3.419237,-0.794693,30.452770


In [10]:
def inter_polation_norm_0_1(x):
    return (x-x.min())/(x.max() - x.min())
def inter_polation_norm_1_1(x):
    return (2*(x-x.min())/(x.max() - x.min()) - 1)
def z_score_norm(x):
    return (x - x.mean())/x.std()
df_rnn = inter_polation_norm_1_1(df_rnn)
df_target = inter_polation_norm_1_1(df_target)
df_rnn = df_rnn[:df_target.shape[0],:,:]
print (df_target.shape)
print (df_rnn.shape)

(4206, 5)
(4206, 30, 19)


In [11]:
## Train test split
x_train, y_train = df_rnn[:3453], df_target.loc[:'2014-01-01'].as_matrix()
x_test, y_test = df_rnn[3454:], df_target.loc['2014-01-01':].as_matrix()

In [12]:
## helper function creating layers

def new_weights(shape, stddev):
    ## Xavier intialization
    initial = tf.truncated_normal(shape=shape, stddev=stddev,dtype=tf.float32)
    
    return tf.Variable(initial)
## Biases initialization
def new_biases(length):
    initial = tf.constant(value=0, shape=[length], dtype=tf.float32)
    return tf.Variable(initial)

def new_layer(in_size, out_size):
    stddev = np.sqrt(np.float(2)/(in_size + out_size))
    weights = new_weights([in_size, out_size], stddev)
    biases = new_biases(out_size)
    return {'weights': weights, 'biases':biases}

In [13]:
## Placeholder variables to hold input data
X = tf.placeholder(tf.float32, [None, temporal_windows, n_features]) # Input data
Y = tf.placeholder(tf.float32, [None, horizons]) # Labels
print ("Input X: {}".format(X))
print ("Target Y: {}".format(Y))

Input X: Tensor("Placeholder:0", shape=(?, 30, 19), dtype=float32)
Target Y: Tensor("Placeholder_1:0", shape=(?, 5), dtype=float32)


In [14]:
layer = new_layer(hidden_size, horizons) # output layer, outmost layer of the network
print ('* Layer: {0}'.format(layer))

# hidden_1_layer = new_layer(hidden_size, 100) # Hidden layer 1, outmost layer of the network
# print ('* Hidden Layer 1: {0}'.format(hidden_1_layer))

# softmax_layer = new_layer(100, N_LABELS) # Softmax layer, outmost layer of the network
# print ('* Hidden Layer 1: {0}'.format(softmax_layer))


# 1 cell LSTM
cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True) # A single LSTM cell

# # Multi LSTM cells
# rnn_cells = tf.contrib.rnn.MultiRNNCell([cell] * n_layers)
# print ('* rnn_cells: {0}'.format(rnn_cells))
# outputs_T, states = tf.contrib.dynamic_rnn(rnn_cells, X_rnn, dtype=tf.float32)
# print ('* output transpose: {0}'.format(outputs_T))

# initial_state = cell.zero_state(batch_size, tf.float32)
print ('* single cell: {0}'.format(cell))
## Single rnn cell

outputs_T, states = tf.nn.dynamic_rnn(cell=cell, inputs=X, dtype=tf.float32)
print ('* outputs transpose: {0}'.format(outputs_T))

outputs = tf.transpose(outputs_T, [1,0,2])
print ('* outputs: {0}'.format(outputs))

# # Use output of last step as input for softmax layer
# last_step = tf.gather(outputs, int(outputs.get_shape()[0]) - 1)
# print ('* last_step: {0}'.format(last_step))
# input_softmax = tf.matmul(last_step, layer['weights']) + layer['biases']
# print ('* input_softmax: {0}'.format(input_softmax))
# y_rnn_softmax = tf.nn.softmax(input_softmax)
# print ('* y_rnn_softmax: {0}'.format(y_rnn_softmax))


## Use mean of output of all steps as input for softmax layer
# mean_step = tf.reduce_mean(input_tensor=outputs, axis=0)
# print ('* mean_step: {0}'.format(mean_step))

# # Hidden layer 1:

# hidden_1 = tf.nn.relu(tf.matmul(mean_step, hidden_1_layer['weights']) + hidden_1_layer['biases'])
# print hidden_1
# input_softmax = tf.matmul(hidden_1, softmax_layer['weights']) + softmax_layer['biases']

input_regression = tf.matmul(outputs[-1], layer['weights']) + layer['biases']
print ('* input_regression: {0}'.format(input_regression))
y_rnn_regression = tf.nn.tanh(input_regression)
print ('* y_rnn_regression: {0}'.format(y_rnn_regression))


* Layer: {'weights': <tf.Variable 'Variable:0' shape=(15, 5) dtype=float32_ref>, 'biases': <tf.Variable 'Variable_1:0' shape=(5,) dtype=float32_ref>}
* single cell: <tensorflow.contrib.rnn.python.ops.core_rnn_cell_impl.BasicLSTMCell object at 0x0000013AB2D22828>
* outputs transpose: Tensor("rnn/transpose:0", shape=(?, 30, 15), dtype=float32)
* outputs: Tensor("transpose_1:0", shape=(30, ?, 15), dtype=float32)
* input_regression: Tensor("add:0", shape=(?, 5), dtype=float32)
* y_rnn_regression: Tensor("Tanh:0", shape=(?, 5), dtype=float32)


In [15]:
LEARNING_RATE = 1e-3
cost = tf.reduce_mean(tf.square(y_rnn_regression-Y))

optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(cost)
## Making Prediction
y_pred = y_rnn_regression
y_true = Y

In [16]:
BATCH_SIZE = 30
TRAINING_EPOCHS = 20000

## Helper function for optimization
def optimize(train_x, train_y, n_epochs, batch_size, session, saver, dir_path):
    n_samples = train_x.shape[0]
    n_iterations = np.int(np.floor(n_samples/batch_size))+1
    start_time = time.time()
    cost_history = np.empty(shape=[1],dtype=float)
    print ("Training.......")
    if not os.path.exists(dir_path):
            os.makedirs(dir_path)
    for epoch in np.arange(n_epochs+1):
        for itr in np.arange(n_iterations):
            start = (itr * batch_size) % (n_samples - batch_size)
            batch_x, batch_y = train_x[start:start + batch_size], train_y[start:start + batch_size]
            feed_dict_train = {X: batch_x, Y: batch_y}
            _, c = session.run([optimizer, cost], feed_dict=feed_dict_train)
            cost_history = np.append(cost_history,c)
            
        if(epoch % 1000 == 0):
            print ("-- Elapsed time -- Epoch -- Cost value -- ")
            print ("-- {:12.6f} -- {:5d} -- {:10.5f} -- ".format((time.time() - start_time), 
                                                                                    epoch, 
                                                                                    c, 
                                                                                    ))
            print ("-- Making prediction at {}th epoch".format(epoch))
            make_prediction(x_test, df_target.loc['2014-01-01':], sess, BATCH_SIZE)
            plt.savefig('{0}{1}th_epoch'.format(dir_path, epoch), dpi=1000)
        
#             Draw weights of convolutional layer
#             if(epoch % (n_epochs/2) == 0):
#                 plot_conv_weights(session, conv_weights[0], 'conv_1', 1, epoch)
#                 plot_conv_weights(session, conv_weights[1], 'conv_2', 1, epoch)
#                 plot_conv_weights(session, conv_weights[2], 'conv_3', 1, epoch)
                  
#         Save model in folder rnn_model
        
#         saver.save(sess, 'rnn_model/new_cnn')
        
#         print running time and output cost value graph
    print ("---Total Running time: %s seconds ---" % (time.time() - start_time))
    print ('*'*50)
    fig = plt.figure(figsize=(10,5))
    plt.clf()
    plt.plot(cost_history)
    plt.show()


## Helper function to print confusion matrix
def make_prediction(test_x, test_y, session, batch_size):
    print ("Making prediction.......")
    start_time = time.time()
    n_samples = test_x.shape[0]
    n_iterations = np.int(np.floor(n_samples/batch_size))+1
    pred = np.zeros((n_samples, test_y.shape[1]))
    true = test_y.as_matrix()
    for itr in np.arange(n_iterations):
        start = (itr * batch_size) % (n_samples - batch_size)
        batch_x = test_x[start:start + batch_size]
        feed_dict_test = {X: batch_x}
        pred[start:start + batch_size] = session.run(y_pred, feed_dict=feed_dict_test)
    pred = reshape_target_array(pred, horizons)
    print (pred.shape)
    true = reshape_target_array(true, horizons)
    print(true.shape)
    rmse = math.sqrt(metrics.mean_squared_error(pred, true))
    mae = metrics.mean_absolute_error(pred, true)
    print ("RMSE = {}".format(rmse))
    print ("MAE = {}".format(mae))
    plt.close()
    plt.clf()
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(range(pred.shape[0]), pred[:,0], color = 'r', label='Predicted Values')
    ax.plot(range(pred.shape[0]), true[:,0], color = 'b', label='Acutal Values')
    ax.legend(loc='upper right', shadow=False)
#     plt.show()
    print ("---Running Time: {0} seconds ---".format((time.time() - start_time)))
    print ('*'*50)



In [17]:
d = './RNN_test_1/'
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    optimize(x_train, y_train, TRAINING_EPOCHS, BATCH_SIZE, sess, saver, d)
    print ("Making prediction on training set")
    make_prediction(x_train, df_target.loc[:'2014-01-01'], sess, BATCH_SIZE)
    print ("Making training prediction on test 1 set")
    make_prediction(x_test, df_target.loc['2014-01-01':], sess, BATCH_SIZE)

Training.......
-- Elapsed time -- Epoch -- Cost value -- 
--     0.691992 --     0 --    0.10933 -- 
-- Making prediction at 0th epoch
Making prediction.......
(756, 1)
(756, 1)
RMSE = 0.37702717784919465
MAE = 0.3174727913821682
---Running Time: 0.11057901382446289 seconds ---
**************************************************
-- Elapsed time -- Epoch -- Cost value -- 
--   639.360679 --  1000 --    0.02874 -- 
-- Making prediction at 1000th epoch
Making prediction.......
(756, 1)
(756, 1)
RMSE = 0.3292922938506078
MAE = 0.2493084997460801
---Running Time: 0.13059234619140625 seconds ---
**************************************************
-- Elapsed time -- Epoch -- Cost value -- 
--  1347.050380 --  2000 --    0.01471 -- 
-- Making prediction at 2000th epoch
Making prediction.......
(756, 1)
(756, 1)
RMSE = 0.32061002627813
MAE = 0.23565985073335607
---Running Time: 0.24067139625549316 seconds ---
**************************************************
-- Elapsed time -- Epoch -- Cost val

KeyboardInterrupt: 